In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
import json  # <-- Needed for converting dict/list objects to JSON strings

def get_place_recording_relationships(place_id, headers, relationship_types=None):
    """
    Fetch all place-recording relationships for a given place ID.

    Args:
        place_id (str): The MusicBrainz ID of the place.
        headers (dict): Headers to be used in the API request.
        relationship_types (list, optional): List of relationship types to filter by.

    Returns:
        list: A list of all relationships for the given place ID.
    """
    if not place_id or len(place_id) != 36:  # Validate UUID format
        print(f"Invalid place ID: {place_id}")
        return []

    BASE_URL = "https://musicbrainz.org/ws/2"
    endpoint = f"{BASE_URL}/place/{place_id}"
    params = {"inc": "recording-rels", "fmt": "json"}

    try:
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        if not data or "relations" not in data:
            print(f"No relations found for place ID: {place_id}")
            return []

        # Fetch all relationships
        relationships = data.get("relations", [])

        # Filter relationships if relationship_types is provided
        if relationship_types:
            relationships = [
                rel for rel in relationships if rel.get("type") in relationship_types
            ]

        return relationships

    except Exception as e:
        print(f"Error fetching place-recording relationships for {place_id}: {e}")
        return []

# --- Example usage: ---

# 1. Read your list of places from a CSV or other source
places_df = pd.read_csv('places.csv')
place_ids = places_df['id'].tolist()

# 2. Define headers as a dictionary
headers = {
    'User-Agent': 'jihbr@umich.edu'
}

# 3. Prepare for checkpointing in Parquet
checkpoint_file = "checkpoint.parquet"
output_data = []

# Try loading an existing checkpoint to skip processed IDs
try:
    checkpoint_df = pd.read_parquet(checkpoint_file)
    processed_place_ids = set(checkpoint_df['place_id'].tolist())
    output_data = checkpoint_df.to_dict('records')  # Continue from checkpoint
    print(f"Loaded {len(output_data)} records from existing checkpoint.")
except (FileNotFoundError, pd.errors.EmptyDataError):
    processed_place_ids = set()

BATCH_SIZE = 1000
count_since_checkpoint = 0

for place_id in tqdm(place_ids, desc="Fetching relationships"):
    if place_id in processed_place_ids:
        continue  # Skip already processed place IDs

    relationships = get_place_recording_relationships(place_id, headers)
    output_data.append({'place_id': place_id, 'relationships': relationships})
    processed_place_ids.add(place_id)
    count_since_checkpoint += 1

    # Check if we've hit our batch threshold for checkpointing
    if count_since_checkpoint >= BATCH_SIZE:
        # Save entire accumulated data so far
        temp_df = pd.DataFrame(output_data)

        # Convert relationships column to JSON strings
        temp_df['relationships'] = temp_df['relationships'].apply(json.dumps)

        temp_df.to_parquet(checkpoint_file, index=False, compression='snappy')
        print(f"Checkpoint saved with {len(output_data)} total records.")
        
        # Reset our counter for the next batch
        count_since_checkpoint = 0

# Finally, save the entire output to a final Parquet file
final_output_df = pd.DataFrame(output_data)

# Convert relationships column to JSON strings for the final write
final_output_df['relationships'] = final_output_df['relationships'].apply(json.dumps)

final_output_df.to_parquet('place_recording_relationships.parquet', 
                           index=False, 
                           compression='snappy')

print(f"Final output saved with {len(output_data)} total records.")


Fetching relationships:   1%|▏         | 1000/69663 [13:46<13:57:11,  1.37it/s]

Checkpoint saved with 1000 total records.


Fetching relationships:   2%|▏         | 1254/69663 [17:07<10:33:17,  1.80it/s]

Error fetching place-recording relationships for c4b1b9a0-6e40-483d-baf3-b4b885eac545: 503 Server Error: Service Temporarily Unavailable for url: https://musicbrainz.org/ws/2/place/c4b1b9a0-6e40-483d-baf3-b4b885eac545?inc=recording-rels&fmt=json


Fetching relationships:   2%|▏         | 1270/69663 [17:17<12:17:37,  1.55it/s]

...

Fetching relationships:  13%|█▎        | 8965/69663 [1:47:34<11:19:49,  1.49it/s]

Error fetching place-recording relationships for d80239fb-fcd7-4dfc-a9b5-b359ab3ad65d: 503 Server Error: Service Temporarily Unavailable for url: https://musicbrainz.org/ws/2/place/d80239fb-fcd7-4dfc-a9b5-b359ab3ad65d?inc=recording-rels&fmt=json


Fetching relationships:  13%|█▎        | 8996/69663 [1:47:54<11:03:11,  1.52it/s]

Error fetching place-recording relationships for 9b9d4dcf-2f74-403e-85a6-d6a472bab7e5: 503 Server Error: Service Temporarily Unavailable for url: https://musicbrainz.org/ws/2/place/9b9d4dcf-2f74-403e-85a6-d6a472bab7e5?inc=recording-rels&fmt=json


Fetching relationships:  13%|█▎        | 9005/69663 [1:48:01<12:19:49,  1.37it/s]